# Basic Statistics for Data Science (with Python)

# 1. Descriptive Statistics and Visualization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
import scipy.stats

import statsmodels.api as sm
import statsmodels.stats.weightstats as sms
import statsmodels.stats.proportion as prop

## Read data

In [ ]:
data_url = "https://raw.githubusercontent.com/nurandi/tlkmathon-basic-stats/main/data/insurance.csv"
insurance_df = pd.read_csv(data_url)

## Basic Stat

In [ ]:
insurance_df.info()

In [ ]:
insurance_df.shape

In [ ]:
insurance_df.head()

In [ ]:
insurance_df.describe()

In [ ]:
insurance_df['age'].mean()

In [ ]:
insurance_df['age'].std()

In [ ]:
insurance_df['age'].median()

In [ ]:
insurance_df['age'].min()

In [ ]:
insurance_df['age'].max()

In [ ]:
insurance_df.groupby('sex').agg({'age':['mean', 'std', 'var', 'count']}).reset_index()

In [ ]:
smoker_count = insurance_df.groupby('sex').agg({'smoker': 'count'}).reset_index()
smoker_count

In [ ]:
smoker_count['percentage'] = 100 * smoker_count.smoker/smoker_count.smoker.sum()
smoker_count

In [ ]:
pd.crosstab(insurance_df.sex, insurance_df.smoker)

In [ ]:
pd.crosstab(insurance_df.sex, insurance_df.smoker, margins=True, margins_name='Total')

## Data Viz

In [ ]:
sex_charges = insurance_df.groupby('sex')[['charges']].mean().reset_index()

In [ ]:
ax = sns.barplot(x='sex', y='charges', data=sex_charges)

In [ ]:
ax = sns.histplot(insurance_df['bmi'])

In [ ]:
ax = sns.scatterplot(x='age', y='charges', data=insurance_df)

In [ ]:
ax = sns.boxplot(y='charges', data=insurance_df)

In [ ]:
ax = sns.boxplot(x='sex', y="charges", data=insurance_df)

# 2. Probability Distribution

**From PPT:** The weights of adult-males are known to be normally distributed with a mean of 70 kgs and a standard deviation of 13 kgs. Find the percentage of adult-males with weights less than 80 kgs

In [ ]:
weight_mean = 70
weight_stdev = 13
x0 = 80

In [ ]:
prob0 = scipy.stats.norm.cdf((x0 - weight_mean)/weight_stdev)
prob0

 Whats is probability that an adult-male has a weight between 70 and 80kg 

In [ ]:
x1 = 60
prob1 = scipy.stats.norm.cdf((x1 - weight_mean)/weight_stdev)
prob = prob0 - prob1
prob

Using the insurance data, what is the probability of having an BMI greater than 40?

In [ ]:
bmi_mean = insurance_df['bmi'].mean()
bmi_stdev = insurance_df['bmi'].std()

x0 = 40
prob0 = scipy.stats.norm.cdf((x0 - bmi_mean)/bmi_stdev)
print(1 - prob0)

In [ ]:
bmi_std = (insurance_df['bmi'] - bmi_mean)/bmi_stdev

In [ ]:
bmi_std_mean = bmi_std.mean()
bmi_std_stdev = bmi_std.std()
print(bmi_std_mean,bmi_std_stdev)

In [ ]:
ax = sns.histplot(bmi_std)

In [ ]:
ax = sns.histplot(insurance_df['bmi'])

# 3. Hypothesis testing

## Confidence Interval

**From PPT:** We measure the heights of 40 randomly chosen men, and get a mean height of 175cm. We have already known that standard deviation is 20cm. Calculate the 95% confidence interval for mean height!


In [ ]:
n = 40
height_mean = 175
height_stdev = 20

In [ ]:
se = height_stdev/sqrt(n)

In [ ]:
scipy.stats.norm.interval(0.95, loc=height_mean, scale=se)

## Hypothesis testing

### One-sample T-test

A premium golf ball production line must produce all of its balls to 1.615 ounces (45.78 grams) in order to get the top rating (and therefore the top dollar). Samples are drawn hourly and checked. If the production line gets out of sync with a statistical significance of more than 1%, it must be shut down and repaired. One hour samples have been drawn.

1.616, 1.610, 1.615, 1.617, 1.618, 1.614, 1.615, 1.617, 1.610, 1.590, 1.610, 1.619, 1.620, 1.611, 1.612, 1.614, 1.615

Do you shut down the line?

Hypothesis:
-   $H_0: µ = 1.615$ ("mean of weight = 1.615 ounces")
-   $H_1: µ ≠ 1.615$

In [ ]:
sample = np.array([1.616, 1.610, 1.615, 1.617, 1.618, 1.614, 1.615, 1.617, 1.610, 1.590, 1.610, 1.619, 1.620, 1.611, 1.612, 1.614, 1.615])

#### Step 1: Calculate number of sample, mean and std. dev

In [ ]:
sample_mean = sample.mean()
sample_stdev = sample.std()
n = sample.size
df = n - 1

In [ ]:
alpha = 0.01
h1_mean = 1.615

#### Step 2: Find/calculate critical value(s)

In [ ]:
# if you have python 
cv1 = scipy.stats.t.ppf(alpha/2, df)
cv2 = scipy.stats.t.ppf(1- (alpha/2), df)
print(cv1, cv2)

Or, simple use t-distribution table

![t-table](https://image.slidesharecdn.com/t-distributiontable-170723122602/95/t-distribution-table-1-638.jpg)

#### Step 3: Calculate t-statistic and/or p-value

In [ ]:
tstat = (sample_mean - h1_mean)/(sample_stdev/(sqrt(df)))
tstat

In [ ]:
pvalue = (1 - scipy.stats.t.cdf(abs(tstat), df)) * 2
pvalue

#### Step 4: Draw a conclusion

**Alternatif 1:** Compare t-stat to critical value. If t-stat > |critical value| *(for two tailed test)* --> reject H0  
**Alternatif 2:** Compare p-value to alpha. If p-value < alpha --> reject H0  

For this case, we cannot reject H0 and conclude that there are not enough reasons to shut the production line down. 

### Statsmodel way

#### One-sample t-test

In [ ]:
d = sms.DescrStatsW(sample)
d.ttest_mean(value=h1_mean, alternative='two-sided')

#### Confidence interval

In [ ]:
d.tconfint_mean()

### One sample T-Test (2)

Using insurance data, check if on average people are overweight.

According to CDC, BMI 30 or higher is considered as overweight, so we can write our hypothesis:
    
-   $H_0: µ = 30$ ("mean of BMI = 30")
-   $H_1: µ > 30$

In [ ]:
d = sms.DescrStatsW(insurance_df['bmi'])
d.ttest_mean(value=30, alternative='larger')

### Two independent sample T-Test

Using insurance data, does smoking affect BMI (body mass index)?

Hyphotesis
-   $H_0: µ_1 = µ_2$ ("there is no difference in BMI between smoker and non-smoker")
-   $H_1: µ_1 ≠ µ_2$ 


In [ ]:
# Levene Test to check equity of variance

smoker    = insurance_df[insurance_df['smoker'] == 'yes']['bmi']
nonsmoker = insurance_df[insurance_df['smoker'] ==  'no']['bmi']

scipy.stats.levene(smoker, nonsmoker, center='mean')

In [ ]:
# P-value > 0.05, we assume equal variance
scipy.stats.ttest_ind(smoker, nonsmoker, equal_var = True)

### Test for proportion

Using isurance data, can we say that the proportion of smokers between male and female are different?

Hypothesis:
-   $H_0: p_m = p_f$ ("proportion of male smokers equal to proportion of female smokers")
-   $H_1: p_m ≠ p_f$ 

In [ ]:
n_male   = insurance_df.sex[insurance_df['sex'] == 'male'].count()
n_female = insurance_df.sex[insurance_df['sex'] == 'female'].count()

n_male_smoker   = insurance_df.sex[insurance_df['sex'] == 'male'][insurance_df['smoker'] == 'yes'].count()
n_female_smoker = insurance_df.sex[insurance_df['sex'] == 'female'][insurance_df['smoker'] == 'yes'].count()

In [ ]:
n_male_smoker

In [ ]:
n_smoker = np.array([n_male_smoker, n_female_smoker])
n_gender = np.array([n_male, n_female])
h1 = 0 
prop.proportions_ztest(count = n_smoker, nobs = n_gender, value=h1)

### ANOVA

In [ ]:
ne = insurance_df[insurance_df['region'] == 'northeast']['charges']                                               
nw = insurance_df[insurance_df['region'] == 'northwest']['charges']                                               
se = insurance_df[insurance_df['region'] == 'southeast']['charges']
sw = insurance_df[insurance_df['region'] == 'southwest']['charges']

scipy.stats.levene(ne, nw, se, sw, center='mean')

**Note:** ANOVA assume equity of variance (homoscedasticity). Because pvalue is so small, this assumption can not be fullfiled, hence ANOVA theoritically can not be used. 

In [ ]:
scipy.stats.f_oneway(ne, nw, se, sw)

**Note**: For two sample test with equal variance, ANOVA and t-test will draw same conclussion

In [ ]:
scipy.stats.ttest_ind(smoker, nonsmoker, equal_var = True)

In [ ]:
scipy.stats.f_oneway(smoker, nonsmoker)

# 4. Association and correlation

### Chi-Squared Test: Association between two categorical variables

Using insurance data, is there an association between gender and smoker?

-   $H_0$ : no association between gender and smoker
-   $H_1$ : there is an association between gender and smoker

In [ ]:
gender_smoker_table = pd.crosstab(insurance_df.sex, insurance_df.smoker)
#gender_smoker_table = pd.crosstab(insurance_df.sex, insurance_df.smoker, margins=True, margins_name='Total')

In [ ]:
scipy.stats.chi2_contingency(gender_smoker_table, correction = True)

### Pearson's r test: Correlation between two numerical variables

Using insurance data, is there an correlation between age and total charges?

-   $H_0 : r = 0$ : no correlation between age and total charges
-   $H_1 : r ≠ 0$ : there is a correlation between age and total charges

In [ ]:
scipy.stats.chi2_contingency(gender_smoker_table, correction = True)

In [ ]:
ax = sns.scatterplot(x='age', y='charges', data=insurance_df)

In [ ]:
scipy.stats.pearsonr(insurance_df['age'], insurance_df['charges'])

In [ ]:
# calculate correlation coefficients across all numerical variables in panda's dataframe
corr = insurance_df.corr()
corr

In [ ]:
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

# 5. Linear regression

### Simple linear regression

Using insurance data, check if age affects total charges and write formula to predict total charges base on age.

In [ ]:
X = insurance_df['age']
# X = insurance_df['age'].values
y = insurance_df['charges']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
model.summary()

From the summary above, we can write a regression equation:
    
$charges = 3165.89 + 257.72*age + error$

Means, every one year increase in age will increase total charge by 257.72

### Multiple linear regression

In [ ]:
reg_dummy   = pd.get_dummies(insurance_df[['region']], drop_first=True)
sex_dummy   = pd.get_dummies(insurance_df[['sex']], drop_first=True)
smoke_dummy = pd.get_dummies(insurance_df[['smoker']], drop_first=True)

In [ ]:
Xs = pd.concat([insurance_df[['bmi', 'children']], reg_dummy, sex_dummy, smoke_dummy], axis=1)
y = insurance_df['charges']
Xs = sm.add_constant(Xs) 

model = sm.OLS(y, Xs).fit()
model.summary()

Assumptions for linear regression:  
- **Linearity**: The relationship between X and the mean of Y is linear.
- **Homoscedasticity**: The variance of residual is the same for any value of X.
- **Independence**: Observations are independent of each other.
- **Normality**: For any fixed value of X, Y is normally distributed.

Interpreting coeficient for dummy variables:  
- region_northwest = -392.3686 : total charge for northwest region is 392.37 lower than northest (reference)
- sex_male = -310.8509 : total charge for males is 310.85 lower than females (reference)
- smoker_yes = 2.366e+04 : total charge for smokers is 23,660 higher than non-smokers